In [1]:
import pandas as pd
# pd.set_option('display.max_columns', 500)

ROWS = 200000

In [2]:
data = pd.read_csv('data/train.csv.gz', compression='gzip',
                   error_bad_lines=False)

In [3]:
data.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [60]:
data['site_name'].size

37670293

In [61]:
testData = pd.read_csv('data/test.csv.gz', compression='gzip',
                   error_bad_lines=False)
testData['site_name'].size

2528243

In [46]:
data.groupby(['site_name', 'posa_continent'])['posa_continent'].size()

site_name  posa_continent
2          3                 136321
6          0                     64
7          1                    325
8          4                   2683
9          1                    240
10         0                    430
11         3                  13871
13         1                   4606
14         1                    532
15         1                    196
16         1                     14
17         1                   2289
18         2                   1846
19         3                    103
20         2                     25
21         1                     59
22         2                    481
23         1                   2676
24         2                   5799
25         2                   1422
26         0                    333
27         2                    423
28         1                    294
29         1                    348
30         4                    729
31         2                     24
32         1                    411
33

In [53]:
data.dtypes

date_time                     object
site_name                      int64
posa_continent                 int64
user_location_country          int64
user_location_region           int64
user_location_city             int64
orig_destination_distance    float64
user_id                        int64
is_mobile                      int64
is_package                     int64
channel                        int64
srch_ci                       object
srch_co                       object
srch_adults_cnt                int64
srch_children_cnt              int64
srch_rm_cnt                    int64
srch_destination_id            int64
srch_destination_type_id       int64
is_booking                     int64
cnt                            int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
hotel_cluster                  int64
dtype: object

In [69]:
from collections import defaultdict
import datetime
from heapq import nlargest
from operator import itemgetter


print('Preparing arrays...')
f = open("data/train.csv", "r")
f.readline()
best_hotels_od_ulc = defaultdict(lambda: defaultdict(int))
best_hotels_search_dest = defaultdict(lambda: defaultdict(int))
best_hotels_search_dest1 = defaultdict(lambda: defaultdict(int))
best_hotel_country = defaultdict(lambda: defaultdict(int))
popular_hotel_cluster = defaultdict(int)
total = 0


while 1:
    line = f.readline().strip()
    total += 1

    if total % 2000 == 0:
        print('Read {} lines...'.format(total))
        break

    if line == '':
        break

    arr = line.split(",")
    book_year = int(arr[0][:4])
    user_location_city = arr[5]
    orig_destination_distance = arr[6]
    srch_destination_id = arr[16]
    is_booking = int(arr[18])
    hotel_country = arr[21]
    hotel_market = arr[22]
    hotel_cluster = arr[23]

    append_1 = 3 + 17*is_booking
    append_2 = 1 + 5*is_booking

    if user_location_city != '' and orig_destination_distance != '':
        best_hotels_od_ulc[(user_location_city, orig_destination_distance)][hotel_cluster] += 1

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '' and book_year == 2014:
        best_hotels_search_dest[(srch_destination_id, hotel_country, hotel_market)][hotel_cluster] += append_1

    if srch_destination_id != '':
        best_hotels_search_dest1[srch_destination_id][hotel_cluster] += append_1

    if hotel_country != '':
        best_hotel_country[hotel_country][hotel_cluster] += append_2

    popular_hotel_cluster[hotel_cluster] += 1

f.close()


Preparing arrays...
Read 2000 lines...


In [ ]:
print('Generate submission...')
now = datetime.datetime.now()
path = 'submission_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
out = open(path, "w")
f = open("../input/test.csv", "r")
f.readline()
total = 0
out.write("id,hotel_cluster\n")
topclasters = nlargest(5, sorted(popular_hotel_cluster.items()), key=itemgetter(1))

while 1:
    line = f.readline().strip()
    total += 1

    if total % 1000000 == 0:
        print('Write {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    id = arr[0]
    user_location_city = arr[6]
    orig_destination_distance = arr[7]
    srch_destination_id = arr[17]
    hotel_country = arr[20]
    hotel_market = arr[21]

    out.write(str(id) + ',')
    filled = []

    s1 = (user_location_city, orig_destination_distance)
    if s1 in best_hotels_od_ulc:
        d = best_hotels_od_ulc[s1]
        topitems = nlargest(5, sorted(d.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    s2 = (srch_destination_id, hotel_country, hotel_market)
    if s2 in best_hotels_search_dest:
        d = best_hotels_search_dest[s2]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])
    elif srch_destination_id in best_hotels_search_dest1:
        d = best_hotels_search_dest1[srch_destination_id]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    if hotel_country in best_hotel_country:
        d = best_hotel_country[hotel_country]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    for i in range(len(topclasters)):
        if topclasters[i][0] in filled:
            continue
        if len(filled) == 5:
            break
        out.write(' ' + topclasters[i][0])
        filled.append(topclasters[i][0])

    out.write("\n")
out.close()
print('Completed!')


In [70]:
best_hotels_od_ulc

defaultdict(<function __main__.<lambda>>,
            {('10388', '1589.1756'): defaultdict(int, {'0': 1}),
             ('10388', '1589.6089'): defaultdict(int, {'87': 1}),
             ('10388', '1589.9186'): defaultdict(int, {'66': 1}),
             ('10388', '1595.5586'): defaultdict(int, {'65': 1}),
             ('10388', '173.089'): defaultdict(int, {'33': 1}),
             ('10388', '173.459'): defaultdict(int, {'17': 1}),
             ('10388', '174.0058'): defaultdict(int, {'70': 1}),
             ('10388', '174.012'): defaultdict(int, {'41': 1}),
             ('10388', '174.0335'): defaultdict(int, {'37': 2}),
             ('10388', '174.0359'): defaultdict(int, {'97': 1}),
             ('10388', '174.0554'): defaultdict(int, {'56': 1}),
             ('10388', '174.0954'): defaultdict(int, {'55': 2}),
             ('10388', '174.1205'): defaultdict(int, {'72': 1}),
             ('10388', '174.1403'): defaultdict(int, {'4': 1}),
             ('10388', '174.4728'): defaultdict(i